In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [11]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('swish')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 28s 81ms/step - loss: 2.0768 - accuracy: 0.2622 - val_loss: 1.7958 - val_accuracy: 0.4338
Epoch 2/50
313/313 [==============================] - 24s 76ms/step - loss: 1.5497 - accuracy: 0.4598 - val_loss: 1.4903 - val_accuracy: 0.4773
Epoch 3/50
313/313 [==============================] - 26s 81ms/step - loss: 1.3953 - accuracy: 0.5240 - val_loss: 1.4678 - val_accuracy: 0.4892
Epoch 4/50
313/313 [==============================] - 24s 75ms/step - loss: 1.2903 - accuracy: 0.5635 - val_loss: 1.4988 - val_accuracy: 0.4639
Epoch 5/50
313/313 [==============================] - 24s 78ms/step - loss: 1.1960 - accuracy: 0.5975 - val_loss: 1.2983 - val_accuracy: 0.5472
Epoch 6/50
313/313 [==============================] - 24s 78ms/step - loss: 1.1279 - accuracy: 0.6207 - val_loss: 1.3482 - val_accuracy: 0.5326
Epoch 7/50
313/313 [==============================] - 26s 82ms/step - loss: 1.0623 - accuracy: 0.6429 - val_loss: 1.1387 - val_accuracy:

In [12]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.8494691848754883, 1.511410117149353, 1.3718115091323853, 1.2757576704025269, 1.1969853639602661, 1.127747893333435, 1.065446138381958, 1.0123978853225708, 0.9685873985290527, 0.9223372340202332, 0.8858275413513184, 0.8398584723472595, 0.8034319281578064, 0.7674315571784973, 0.7310449481010437, 0.6923319101333618, 0.6601558923721313, 0.6258076429367065, 0.5918991565704346, 0.5576987862586975, 0.530364990234375, 0.4975295066833496, 0.46858182549476624, 0.44097432494163513, 0.41258084774017334, 0.38244596123695374, 0.36341267824172974, 0.34004470705986023, 0.3108637034893036, 0.28961530327796936, 0.26374199986457825, 0.2454206496477127, 0.23235930502414703, 0.21943755447864532, 0.2007129043340683, 0.18528299033641815, 0.17471717298030853, 0.16075444221496582, 0.1509486585855484, 0.1398552805185318, 0.12568044662475586, 0.11941149830818176, 0.10592155903577805, 0.09993626922369003, 0.09875503927469254, 0.08746957778930664, 0.08378904312849045, 0.07695432007312775, 0.07361080497